# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
%%time
data = run_engine(data)

pl    0.00236
dtype: float64
--------------------------------------
             DateTime_frmt          close_type       pl
101469 2021-01-04 18:39:22  simple_take_profit  0.00026
102377 2021-01-04 18:55:44  simple_take_profit  0.00022
103150 2021-01-04 19:07:03  simple_take_profit  0.00020
104686 2021-01-04 19:35:59  simple_take_profit  0.00011
105400 2021-01-04 19:50:55  simple_take_profit  0.00017
105865 2021-01-04 19:57:29  simple_take_profit  0.00025
106346 2021-01-04 20:03:06  simple_take_profit  0.00026
107726 2021-01-04 20:28:54         simple_stop -0.00030
108741 2021-01-04 20:48:57  simple_take_profit  0.00023
110218 2021-01-04 21:21:12  simple_take_profit  0.00031
110690 2021-01-04 21:40:37  simple_take_profit  0.00034
111449 2021-01-04 22:02:39  simple_take_profit  0.00039
111491 2021-01-04 22:06:19  simple_take_profit  0.00051
111647 2021-01-04 22:18:55  simple_take_profit  0.00048
112736 2021-01-04 23:29:33  simple_take_profit  0.00034


100%|███████████████████████████████| 113572/113572 [00:10<00:00, 10705.19it/s]


Wall time: 1min 6s


In [3]:
generate_result_report(data)

--------------------------------------
Run Complete : data/ema_df-(2021-2021)-(1-1)-(1-4).csv
--------------------------------------


--------------------------------------
Net PL
0.0024
--------------------------------------


--------------------------------------
Monthly PL
                pl
month_val         
1          0.00236
--------------------------------------


--------------------------------------
Daily PL
               pl
date_val         
3        -0.00104
4         0.00340
--------------------------------------


--------------------------------------
Count of transactions by Close Type
..................
                     pl
close_type             
simple_stop          54
simple_take_profit  106
--------------------------------------


--------------------------------------
Mean value of transactions by Close Type
..................
                          pl
close_type                  
simple_stop        -0.000452
simple_take_profit  0.000253
----------------

In [74]:
data["plot"] = True
data['reduce_plot'] = False
data['plot_type'] = 'file'
data['plot_start']  = '2021-01-06'
data['plot_stop']   = '2021-01-07'

plot_graph(data)

In [ ]:
# data['df'] = temp

temp = data['df']
temp.head(2)

i = 10000
data['avg_candle_num'] = 3

ind = data['df']['h'][data['df']['h'].notnull()].loc[i - data['candle_size'] * data['avg_candle_num'] + 1 : i].index

data['df']['h_avg'].loc[i:i+data['candle_size']] = np.mean(data['df']['h'].loc[ind] - data['df']['lema'].loc[ind])
data['df']['l_avg'].loc[i:i+data['candle_size']] = np.mean(data['df']['lema'].loc[ind] - data['df']['l'].loc[ind])

data['df']['h']         = np.nan
data['df']['l']         = np.nan
data['df']['h_l_gap']   = np.nan

data['df']['h_avg']     = np.nan
data['df']['l_avg']     = np.nan    

data['df']['h_gap']     = np.nan
data['df']['l_gap']     = np.nan

data['df']['h_lema']    = np.nan
data['df']['l_lema']    = np.nan

for i in tqdm(range(len(data['df']))):
    if i % data['candle_size'] == 0 and i > 0:        
        data['tick_list'] = data['df']['tick'].loc[i - data['candle_size'] : i-1]
        max_val     = max(data['tick_list'])
        min_val     = min(data['tick_list'])

        data['df']['h'].loc[i]  = max_val
        data['df']['l'].loc[i]  = min_val

        ind = data['df']['h'][data['df']['h'].notnull()].loc[i - data['candle_size'] * data['avg_candle_num'] + 1 : i].index
        data['df']['h_gap'].loc[i:i+data['candle_size']] = np.mean(data['df']['h'].loc[ind] - data['df']['lema'].loc[ind])
        data['df']['l_gap'].loc[i:i+data['candle_size']] = np.mean(data['df']['lema'].loc[ind] - data['df']['l'].loc[ind])

data['df']['h_lema'] = data['df']['h_gap'] + data['df']['lema']
data['df']['l_lema'] = data['df']['lema'] - data['df']['l_gap']

data['df']['h_l_gap'] = data['df']['h'] - data['df']['l']

data['df']